In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import  cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC #导库
data=pd.read_csv("E:/text/train.csv",low_memory=False) #读入数据
test=pd.read_csv("E:/text/test.csv",low_memory=False)
data.dropna(inplace=True)#删除空值
#删除重复值或者完全不同的值
def del_features(data):
    for name in data.columns:
        unique = data[name].unique().shape[0]
        full = data[name].shape[0]
        if (unique == 1) | (unique == full):
            data = data.drop(name, axis=1)
    return data
del_features(data)
y_train=data['bad_good']#保留data的标签，以免后面变动
data.drop('bad_good',axis=1,inplace=True)#删除data的标签
data=pd.concat([data,test],axis=0) #合并data和test
data_num=data.select_dtypes(include=[np.number])#找出数字列
data_non_num=data.select_dtypes(exclude=[np.number])#找出文本列
data_non_numDf=pd.get_dummies(data_non_num)   #进行one-hot编码
data_x=pd.concat([data,data_non_numDf],axis=1) #one-hot编码后数据

data_x.drop([
       'IDF_TYP_CD', 
       'GENDER', 
       'CUST_EUP_ACCT_FLAG',
       'CUST_AU_ACCT_FLAG', 
       'CUST_SALARY_FINANCIAL_FLAG', 
       'CUST_SOCIAL_SECURITYIC_FLAG',
       'CUST_MTFLOW_FLAG',
       'CUST_DOLLER_FLAG',
       'CUST_INTERNATIONAL_GOLD_FLAG',
       'CUST_INTERNATIONAL_COMMON_FLAG', 
       'CUST_INTERNATIONAL_SIL_FLAG',
       'CUST_INTERNATIONAL_DIAMOND_FLAG', 
       'CUST_GOLD_COMMON_FLAG',
       'CUST_STAD_PLATINUM_FLAG', 
       'CUST_LUXURY_PLATINUM_FLAG',
       'CUST_PLATINUM_FINANCIAL_FLAG',
       'CUST_DIAMOND_FLAG',
       'CUST_INFINIT_FLAG',
       'CUST_BUSINESS_FLAG',
       'DEP_SA_FLAG',
       'DEP_TD_FLAG',
       'LOAN_FLAG',
       'FUND_FLAG', 
       'C_FUND_FLAG',
       'D_FUND_FLAG',
       'S_FUND_FLAG',
       'BOND_FLAG', 
       'CER_BOND_FLAG',
       'BK_BOND_FLAG', 
       'CRED_FLAG',
       'RELATED_REPAY_FLAG',
       'TOT_REPAY_FLAG'
],axis=1,inplace=True)#删除无意义列
data_x.drop('CUST_ID',axis=1,inplace=True)#删除没意义CUST_ID列

from sklearn.feature_selection import VarianceThreshold#删除方差为0的特征
X_fsvar = VarianceThreshold().fit_transform(data_x)

x_train=X_fsvar[0:285285,:] #提取train部分
x_test=X_fsvar[285285:,:]

from sklearn.model_selection import train_test_split

# 划分数据集 
x_train, x_test, y_train, y_test = train_test_split(
    x_train, y_train, test_size=0.3
)

score=cross_val_score(RFC(n_estimators=100),x_train,y_train,cv=5).mean()


rfc=RFC(n_estimators=100).fit(x_train,y_train)
y1=rfc.predict(x_test)
y1=pd.DataFrame(y1)
y1['bad_good']=y1
y1.drop(0,axis=1,inplace=True)
submission=pd.concat([test['CUST_ID'],y1],axis=1)
submission.to_csv("E:/text/submission.csv",index=0)
